# 16. Natural Language Processing with RNNs and Attention

Looking at it from a certain perspective, the Turing test is an NLP task. This chapter will focus on how to tackle NLP tasks (albeit less complex than a Turing test) using RNNs. 

### Generating Shakespearean Text Using a Character RNN

Let's look at how to build a Char-RNN, a net that predicts the next character in a sentence. 

#### Creating the Training Dataset

Downloading the file from Andrej Karpathy's GitHub repo:

In [1]:
from tensorflow import keras
import os

filepath = os.path.join(os.getcwd(), 'datasets', 'shakespeare', 'input.txt')
with open(filepath) as f:
    shakespeare_text = f.read()

Next, we must encode every character as an integer. We will use `Tokenizer` for this. 

In [2]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

Let's quickly check what it does: 

In [3]:
tokenizer.texts_to_sequences(["Hello"])

[[7, 2, 12, 12, 4]]

In [4]:
tokenizer.sequences_to_texts([[7, 2, 12, 12, 4]])

['h e l l o']

In [5]:
max_id = len(tokenizer.word_index) # number of distinct characters

In [6]:
dataset_size = tokenizer.document_count # total number of characters

In [8]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # starting from 0

#### How to Split a Sequential Dataset

It is very important to avoid any overlap between the training set, the validation set, and the test set. It would also be a good idea to leave a gap between these sets to avoid the risk of a paragraph overlapping over two sets.

In our case, let's keep 90% for the training set:

In [10]:
import tensorflow as tf

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

#### Chopping the Sequential Dataset into Multiple Windows

Now we have a single very long sequence of characters. We can't just train our RNN on it. Let's use `window` to to convert this long sequence of characters into many smaller windows of text.

In [11]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

By default, windows are **not** overlapping, but we used `shift=1` to make them so. We drop remainder to keep all the windows to 101 character length. 

In [12]:
# flattening our dataset
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Now the dataset contains consecutive windows of 101 characters each. To get the best out of Gradient Descent, we can batch the windows and then separate the inputs (first 100 chars) from the target (last char). 

In [13]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Let's encode each character: 

In [14]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id),
Y_batch))

In [15]:
# prefetching
dataset = dataset.prefetch(1)

### Sentiment Analysis

IMBd is the MNIST of NLP. Let's play around with it in this section. 

In [22]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 8s 0us/step


In [23]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

The reviews are already tokenized and indexed by frequency (low = most frequent). Special integers:

0 = Padding token  
1 = Start of sequence token  
2 = Unkown words  

Let's visualize a review: 

In [24]:
word_index = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 3s 2us/step


In [25]:
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}

In [26]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [27]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

Although tokeninizing by word may work well in English (at least in most cases) this may create issues in other languages (i.e. 中文). 

Next, let's move on doing preprocessing ourselves in TF. Let's load the reviews as strings: 

In [28]:
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True,
                            with_info=True)
train_size = info.splits["train"].num_examples

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\giuse\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [29]:
# preprocessing function

def preprocess(X_batch, y_batch):
    # keeping only first 300 chars
    X_batch = tf.strings.substr(X_batch, 0, 300)
    # turn <br \> into space
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    # anything that is not a letter becomes a space
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

Next, we need to construct the vocabulary.

In [31]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [32]:
# most common words
vocabulary.most_common()[:3]

[(b'<pad>', 215221), (b'the', 61137), (b'a', 38564)]

In [33]:
# keeping only 10000 most popular 
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [34]:
# replace each word with ID
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
# out of vocabulary (oov)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [35]:
table.lookup(tf.constant([b"This movie was amaaaazing".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10698]], dtype=int64)>

Note how the last word goes into an oov bucket (ID > 10000). 

In [37]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

Time to create the model and train it!

In [38]:
embed_size = 128

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 126s 162ms/step - loss: 0.5859 - accuracy: 0.6523
Epoch 2/5
782/782 [==============================] - 117s 150ms/step - loss: 0.3590 - accuracy: 0.8415
Epoch 3/5
782/782 [==============================] - 96s 122ms/step - loss: 0.2271 - accuracy: 0.9103
Epoch 4/5
782/782 [==============================] - 101s 129ms/step - loss: 0.1247 - accuracy: 0.9556
Epoch 5/5
782/782 [==============================] - 109s 139ms/step - loss: 0.0784 - accuracy: 0.9738
